In [49]:
from flask import Flask, render_template, request
import requests
from flask import Flask
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import os
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
import json
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
app = Flask(__name__)
model_name = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [43]:
def get_dietary_advice(text):
  url = "https://nutrition-by-api-ninjas.p.rapidapi.com/v1/nutrition"
  querystring = {"query":text}
  headers = {
    "X-RapidAPI-Key": "33ee5f19c7msh22e43b81068cb1dp18a9adjsnf084b89fcb58",
    "X-RapidAPI-Host": "nutrition-by-api-ninjas.p.rapidapi.com"
  }
  response = requests.get(url, headers=headers, params=querystring)
  data = response.json()
  return data
  pass

In [44]:
def extract_intent(text):
    intent = {}
    # Keywords for each intent category
    type_keywords = ['cardio', 'olympic_weightlifting', 'plyometrics', 'powerlifting', 'strength', 'stretching', 'strongman']
    muscle_keywords = ["abdominals", "abductors", "adductors", "biceps", "calves", "chest", "forearms", "glutes", "hamstrings", "lats", "lower_back", "middle_back", "neck", "quadriceps", "traps", "triceps"]
    difficulty_keywords = ['beginner', 'intermediate', 'expert']
    # Extracting keywords for 'type' intent
    type_intent = [keyword for keyword in type_keywords if keyword in text]
    if type_intent:
        intent['type'] = type_intent
    # Extracting keywords for 'muscle' intent
    muscle_intent = [keyword for keyword in muscle_keywords if keyword in text]
    if muscle_intent:
        intent['muscle'] = muscle_intent
    # Extracting keywords for 'difficulty' intent
    difficulty_intent = [keyword for keyword in difficulty_keywords if keyword in text]
    if difficulty_intent:
        intent['difficulty'] = difficulty_intent
    return intent
    pass

In [45]:
def get_workout_recommendation(text):
    querystring = extract_intent(text)
    if not querystring :
        print("Sorry, the instructions are not clear")
        return None
    url = "https://exercises-by-api-ninjas.p.rapidapi.com/v1/exercises"
    headers = {
        "X-RapidAPI-Key": "33ee5f19c7msh22e43b81068cb1dp18a9adjsnf084b89fcb58",
        "X-RapidAPI-Host": "exercises-by-api-ninjas.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()
    return data[0]

In [ ]:
def generate_response(text):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    output = model.generate(input_ids, max_length=50, num_return_sequences=1, early_stopping=True)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    dapi_response = str(get_dietary_advice(text))
    Wapi_response=''
    if not get_dietary_advice(text):
      Wapi_response = str(get_workout_recommendation(text))
    response = generated_text + " " + Wapi_response + ' ' + dapi_response
    return str(response)
@app.route("/")
def home():
    return render_template("index.html")
@app.route("/get")

In [50]:
def get_bot_response():
  text = request.args.get('msg')
  response = generate_response(text)
    return response
if __name__ == "__main__":

IndentationError: unexpected indent (<ipython-input-50-e73a03651467>, line 4)

In [48]:
messages = [{"role": "system", "content": "You are an intelligent assistant."}]
while True:
    message = input("User: ")
    if message =="hi":
      print("Your gym bro: Hey dude! How can I help you as your gym bro")
    elif message == 'bye':
      print("Your gym bro: bye! keep hustlin' ")
      break
    elif message:
        messages.append({"role": "user", "content": message})
        reply = generate_response(message)
        print(f"Your gym bro: {reply}")
        messages.append({"role": "assistant", "content": reply})
if __name__ == "__main__":

User: train my biceps as per beginner
Your gym bro: train my biceps as per beginner {'name': 'Incline Hammer Curls', 'type': 'strength', 'muscle': 'biceps', 'equipment': 'dumbbell', 'difficulty': 'beginner', 'instructions': 'Seat yourself on an incline bench with a dumbbell in each hand. You should pressed firmly against he back with your feet together. Allow the dumbbells to hang straight down at your side, holding them with a neutral grip. This will be your starting position. Initiate the movement by flexing at the elbow, attempting to keep the upper arm stationary. Continue to the top of the movement and pause, then slowly return to the start position.'} []
User: bye
Your gym bro: bye! keep hustlin' 
